# Description

__Цель__

Хотим понять места в приложении, которые непонятны пользователю и провоцируют удаление аккаунта.

__Задачи__

1. Собрать необходимые данные
2. Подготовить данные для аналитики
3. Провести анализ
    1. Построить сводные таблицы
    2. Визуализировать траектории
    3. Построить классификатор
        1. Помогает выделить специфичные для конкретной группы траектории
        2. Позволяет по текущей траектории оценить вероятность удаление аккаунта пользователя, например, чтобы динамически менять контент приложения, чтобы люди оставались

__Ожидаемые результаты__

1. Узнать наиболее проблемные участки приложения, чтобы предотвратить уход юзеров
2. Получить классификатор, который позволит предсказывать удаление аккаунта по тому, как пользователь использует приложение

# Download data

In [ ]:
import numpy as np
import retentioneering
from retentioneering.utils import download_events_multi, preparing

In [ ]:
client, job_config = retentioneering.init_from_json('./settings.json')
settings = retentioneering.Config('./settings.json')

#### you can use one settings file in download_events_multi for multiple queries

In [ ]:
df = download_events_multi(client, job_config=job_config, settings=settings)
print 'Downloaded DataFrame shape: {}'.format(df.shape)

#### It creates a new column 'group_name' with name of the query

In [ ]:
df.group_name.unique()

In [ ]:
path = '../../data/data_from_bq_delete_accounts.csv'
df.to_csv(path, sep=';', encoding='utf-8', index=False)

# Preparing

## Now you can prepare your DataFrame for further analysis using pandas instruments

#### Drop duplicated events

In [ ]:
df = preparing.drop_duplicated_events(df, settings=settings)

#### delete from test group users from deleted group

In [ ]:
# how many users we have in each group
df.groupby('group_name').user_pseudo_id.nunique()

In [ ]:
# split in two groups
df_deleted = df.loc[df.group_name == u'deleted', :].copy()
df_test = df.loc[df.group_name == u'test_group', :].copy()

In [ ]:
# select users from 'deleted' group
selected_users = df_deleted.user_pseudo_id.unique()

# drop selected users from second group
df_test = df_test.loc[~df_test.user_pseudo_id.isin(selected_users)]

In [ ]:
print 'deleted:', df_deleted.user_pseudo_id.nunique()
print 'test_group:', df_test.user_pseudo_id.nunique()

#### leave only users who delete their account in first two days and has 'first_open' event

In [ ]:
# leave only users with first open
has_first_open_users = df_deleted.loc[df_deleted.event_name == u'first_open', 'user_pseudo_id'].unique()
df_deleted = df_deleted.loc[df_deleted.user_pseudo_id.isin(has_first_open_users), :]
print 'deleted group users:', df_deleted.user_pseudo_id.nunique()

In [ ]:
# for each user find timestamp when account was deleted, and timestamp of first event
delete_account_time = df_deleted.groupby('user_pseudo_id', as_index=False)['event_timestamp'].max()
first_event_time = df_deleted.groupby('user_pseudo_id', as_index=False)['event_timestamp'].min()

# calculate users lifetime
users_lifetime = first_event_time.merge(delete_account_time, 
                                        how='left', 
                                        on='user_pseudo_id', 
                                        suffixes=('_min', '_del'))

In [ ]:
days_threshold = 2
# select users with timedelta 'days_threshold' days and leave only them
selected_users = users_lifetime[(users_lifetime['event_timestamp_del'] -
                users_lifetime['event_timestamp_min']) / 1e6 / (24*60*60) <= days_threshold].user_pseudo_id.unique()
df_deleted = df_deleted.loc[df_deleted.user_pseudo_id.isin(selected_users), :]
print 'deleted group users:', df_deleted.user_pseudo_id.nunique()

In [ ]:
# in second group leave only events within two days
first_event_time = df_test.groupby('user_pseudo_id', as_index=False)['event_timestamp'].min()
df_test = df_test.merge(first_event_time, how='left', on='user_pseudo_id', suffixes=('', '_min'))

In [ ]:
df_test = df_test.loc[(df_test.event_timestamp - df_test.event_timestamp_min) / 1e6 / (24*60*60) <= days_threshold, :]

In [ ]:
# randomly select same size of users from test dataset
np.random.seed(seed=42)
df = df_test.loc[df_test.user_pseudo_id.isin(
    np.random.choice(df_test.user_pseudo_id.unique(), size=df_deleted.user_pseudo_id.nunique(), replace=False)), :] \
    .append(df_deleted, sort=False)

In [ ]:
path = '../../data/data_from_bq_delete_accounts_prepared_data.csv'
df.to_csv(path, sep=';', encoding='utf-8', index=False)

# Analytics

In [ ]:
import pandas as pd
from retentioneering import analysis
from retentioneering.utils import preparing
path = '../../data_from_bq_delete_accounts_prepared_data.csv'
df = pd.read_csv(path, sep=';', encoding='utf-8')

In [ ]:
# add accumulator events

df = preparing.add_lost_events(
    df, positive_event_name=u'settings_delete_account_success', negative_event_name=u'not_delete_account')

In [ ]:
# load filters for events
event_filter = pd.read_csv('~/NewUserEventList.csv').values
lexa_filter = pd.read_csv('~/lexa_filter.csv', ';')
lexa_filter = lexa_filter[lexa_filter.Created_by == '1']['Event Action'].values
lf = {'app_provisional_enabledPush', 'app_enabledPush','app_enabledRemotePush'}
event_filter = set(lexa_filter)|set(event_filter[:, 0])|{'settings_delete_account_success', 'not_delete_account'}
df = df[df.event_name.isin(event_filter - lf)]

In [ ]:
# take only top-30 events for better visualization
top_events = df.groupby('event_name').user_pseudo_id.count()
top_events = set(top_events.sort_values().iloc[-20:].index)|set(['settings_delete_account_success', 'not_delete_account'])
df = df[df.event_name.isin(top_events)]

In [ ]:
desc = analysis.get_desc_table(df, target_event_list=['settings_delete_account_success',
                                                      'not_delete_account'],
                               max_steps=30, settings=settings, plot=True)

In [ ]:
lost_users_list = df[df.event_name == 'settings_delete_account_success'].user_pseudo_id
filt = df.user_pseudo_id.isin(lost_users_list)
df_lost = df[filt]
df_passed = df[~filt]

desc_loss = analysis.get_desc_table(df_lost, target_event_list=['settings_delete_account_success',
                                                                'not_delete_account'],
                                    max_steps=30, settings=settings, plot=False)
desc_passed = analysis.get_desc_table(df_passed, target_event_list=['settings_delete_account_success',
                                                                    'not_delete_account'],
                                      max_steps=30, settings=settings, plot=False)

In [ ]:
diff_df = analysis.get_diff(desc_loss, desc_passed, settings=settings, precalc=True)

Plot graph with python

In [ ]:
df_agg = analysis.get_all_agg(df_lost, agg_list=['trans_count'])
analysis.utils.plot_graph_python(df_agg, 'trans_count', settings)

Or plot graph via api

`It sends your data on our server`

In [ ]:
from retentioneering.utils.export import plot_graph_api
plot_graph_api(df_lost, settings, task='delete')

In [ ]:
# get raw data for model
df = pd.read_csv(path, sep=';', encoding='utf-8')

In [ ]:
# fit model
clf = analysis.Model(df, target_event='settings_delete_account_success', event_filter=event_filter, settings=settings)
clf.fit_model()

We can find leaky-events by analysing model output

In [ ]:
importance = clf.model.coef_[0]
names = clf._embedder.inverse_transform([importance])[0]
importance = importance[importance != 0]
list(zip(names, importance))

We can see that `settings_delete_account` and `settings` has much more importance then other, so we should add it to filter

In [ ]:
event_filter = event_filter|set(['settings_delete_account', 'settings'])
# and rebuild model
clf = analysis.Model(df, target_event='settings_delete_account_success', event_filter=event_filter, settings=settings)
clf.fit_model()

In [ ]:
# repeat it
importance = clf.model.coef_[0]
names = clf._embedder.inverse_transform([importance])[0]
importance = importance[importance != 0]
list(zip(names, importance))

In [ ]:
event_filter = event_filter|set(['profile_settings'])
clf = analysis.Model(df, target_event='settings_delete_account_success', event_filter=event_filter, settings=settings)
clf.fit_model()

In [ ]:
# one more time
importance = clf.model.coef_[0]
names = clf._embedder.inverse_transform([importance])[0]
importance = importance[importance != 0]
list(zip(names, importance))

Now all weights seems more uniformly distributed, so we can stop.

We also can get most valued edges and nodes from model to visualize it on graph.

In [ ]:
imp_tracks = clf.build_important_track()
# edges
imp_tracks[imp_tracks[1].notnull()]

In [ ]:
# nodes
imp_tracks[imp_tracks[1].isnull()][0].values